# Исследование надёжности заёмщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
# импорт библиотек
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

# библиотека для лемматизации
from pymystem3 import Mystem
m = Mystem()

## Изучение данных

В первую очередь изучим имеющиеся данные:
- получим 10 первых и 10 последних строк
- получим общую информацию о датасете и сформулируем первые выводы

In [2]:
# чтение данных и получение первых 10 строк
data = pd.read_csv('/Users/danilvlasenko/Desktop/projects_datasets/data_borrowers.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# получение последних 10 строк датасета
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [4]:
# получение общей информации о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# применение метода .describe() к датасету
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В имеющемся датасете всего 12 столбцов и 21525 строк. Наименования столбцов:

- `children` — количество детей в семье.
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Наименования столбцов корректные, в их переименовании нет необходимости.

Предварительно, в датасете существуют следующие ошибки:
- 1) в столбцах `days_empolyed` и `total_income` присутствуют пропущенные значения;
- 2) в столбце `days_employed` тип данных *float* вместо *integer*;
- 3) в столбце `days_employed` присутствуют отрицательные значения;
- 4) в столбце `education` данные прописаны в различном регистре;
- 5) в столбце `total_income` суммы не округлены;
- 6) в столбце `family_status` присутствуют задвоения (женат/замужем == гражданский брак)

Выведена статистика по столбцам с числовыми значения float и int.
Здесь подтверждаются п. 1, 2, 3, 5 поиска предварительных ошибок. Дополнительно, сразу заметны аномальные значения столбцов children (-1 и 20) и days_employed (1100 лет стажа).

Проверим количество аномальных значений стажа в днях, при необходимости поделим их на 365 или снизим порядок. Если таких значений окажется мало на фоне всего датасета, появится опция их удаления - точность анализа не пострадает. Возможно, при пополнении данных была допущена ошибка, и банку стоит выдать задачу своим DE по проверке источника данных.

## Предобработка данных

### Обработка пропусков

При обработке пропусков отработаем пункты 1 и 3 предварительного анализа.

Сперва поймем, какой процент от всего датасета составляют строки с пропущенными значениями. Если процент небольшой (не больше 5) - их можно удалить. Если процент больше, то стоит их заполнить - в противном случае пострадает точность анализа.

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Строк с пропущенными значениями 2174. Это явно больше 5% от 21525, поэтому столбцы `days_empolyed` и `total_income` заполним и перезапишем в текущую переменную с данными.
- Переведем значения столбца `days_empolyed` в положительные;
- Пропущенные значения в обоих столбцах заполним *медианными* значениями. В случае средних возможен перевес в меньшую или большую стороны.

In [7]:
# замена отрицательных значений
data['days_employed'] = data['days_employed'].abs()

# проверка устранения отрицательных значений
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Сразу же перед расчетом медиан и заполнением пропусков заменим аномальные значения в трудовом стаже (столбец `days_employed`)

Допустим, максимальный адекватный трудовой стаж - 52 года (человек работает постоянно с 18 до 70 лет). Предположим, что в году 240 рабочих дней (в среднем 20 р.д. в месяц). Это 12480 дней. 
Проверим количество абсолютных значений, превышающих 12480. Помним, что перевод в абсолютные значения произведен в предыдущей ячейке.

In [8]:
# подсчет количества аномальных значений трудового стажа
data[data['days_employed'] > 12480]['days_employed'].count()

3515

Таких значений 3515 - слишком много для удаления. Для адекватности значений заменим все 3515 значений на адекватный максимум 12480 дней.

In [9]:
# подготовка функции для замены на адекватный максимум

# функция проходит по всем уникальным значениям столбца days_employed.
# в случае выхода значения за адекватный предел, значение принудительно заменяется на максимально адекватное.
def experience_replace(days_employed_before, days_employed_max):
    for days in days_employed_before:
        if days > 12480:
            data['days_employed'] = data['days_employed'].replace(days, days_employed_max)

In [10]:
# преобразование значений столбца days_employed в список
days_employed_before = data['days_employed'].unique().tolist()
days_employed_max = 12480

# применение функции
experience_replace(days_employed_before, days_employed_max)

In [11]:
# проверка количества строк со значением столбца days_employed == 12480
data[data['days_employed'] == 12480]['days_employed'].count()

3515

In [12]:
# проверка методом .describe().
# максимальное значение в столбце должно быть 12480 и среднее сильно меньше, чем 63046 (было ранее)
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,4150.512627,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,4389.675203,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,12480.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь рассчитаем медианные значения по трудовому стажу и уровню дохода клиентов.

In [13]:
# получение медианных значений для столбцов days_employed и total_income
days_employed_median = data['days_employed'].median()
total_income_median = data['total_income'].median()
print('Медианный трудовой стаж: {:.0f} дней, в годах: {:.2f} лет'.format(days_employed_median, days_employed_median / 365))
print('Медианный доход клиента: {:.0f} руб., или {:.2f} тыс. руб.'.format(total_income_median, total_income_median / 1000))

Медианный трудовой стаж: 2194 дней, в годах: 6.01 лет
Медианный доход клиента: 145018 руб., или 145.02 тыс. руб.


Медианные значения в целом адекватные, ими можно заполнить пропуски в данных.

In [14]:
# заполнение столбцов медианными значениями
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)

In [15]:
# проверка заполнения
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод по обработке пропусков**

Пропущенные значения в столбцах `days_empolyed` и `total_income` устранены, строки с пропущенными значениями в датасете отсутствуют. Значения столбца `days_empolyed` переведены в положительные.

Пункты 1 и 3 предварительного анализа отработаны.

Появление пропущенных значений может быть обусловлено двумя факторами:
* человеческим - возможно, не все желающие получить кредит готовы делиться своим стажем и уровнем дохода;
* технологиическим - в форме заполнения заяки на кредит эти поля не были обязательными для заполнения. Исходя из предыдущего пункта, клиенты их не торопились заполнять.

### Замена типов данных

На этапе замены отработаем пункты 2, 4 и 5 предварительного анализа:
- заменим тип данных на *integer* в столбце `days_employed` - не совсем корректно говорить о стаже в 123,4 дня;
- Для удобства, в столбце `total_income` также изменен на *integer*;
- изменим регистр в столбце `education` - в будущем возможна какая-либо агрегация по уровню образования.

In [16]:
# замена типа данных в столбце days_employed на int
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')

In [17]:
# изменение регистра в столбцах education и family_status
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

In [18]:
# проверка работы изменения регистра
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [19]:
# проверка изменения типа данных в столбце days_employed
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод по замене типов данных**

Тип данных в столбцах `days_employed` и `total_income` заменен на *int64*, регистр в столбце `education` изменен.

Пункты 2, 4 и 5 предварительного анализа устранены.

### Обработка дубликатов

В датасете возможно присутствие явных и неявных дубликатов.

На предыдущем этапе регистр столбца `education` уже был изменен, неявные дубликаты в целом по строке могли превратиться в явные.

In [20]:
# подсчет количества явных дубликатов в датасете
data.duplicated().sum()

71

Явные дубликаты есть, но их совсем немного. Их удаление не скажется на точности анализа.

In [21]:
# удаление явных дубликатов и с перезаписью индексов строк
data = data.drop_duplicates().reset_index(drop=True)

In [22]:
# проверка устранения явных дубликатов
data.duplicated().sum()

0

Проверим столбцы `education`, `family_status`, `income_type` и `purpose` на наличие неявных дубликатов.

In [23]:
# проверка столбца education, по алфавиту
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [24]:
# проверка столбца family_status, по алфавиту
data['family_status'].sort_values().unique()

array(['в разводе', 'вдовец / вдова', 'гражданский брак',
       'женат / замужем', 'не женат / не замужем'], dtype=object)

In [25]:
# проверка, стоит ли объединять статусы 'гражданский брак' и 'женат / замужем'
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [26]:
# проверка столбца purpose, по алфавиту
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Выводы по каждому столбцу:
- 1) В столбце `education` неявных дубликатов нет;
- 2) В столбце `income_type` неявных дубликатов нет;
- 3) В столбце `family_status` статус 'гражданский брак' по смыслу дублируется с 'женат/замужем'. При анализе мысленно будем считать их как один статус, при замене этих дубликатов функцией слетит нумерация в столбце family_status_id;
- 4) В столбце `purpose` дубликаты есть. Но заменять одним значением стоит при лемматизации по типу 'жилье', 'образование', 'автомобиль' и т.п. 

**Вывод по обработке дубликатов**

Датасет проверен на явные и неявные дубликаты.


Явные дубликаты устранены, среди неявных дубликатов внесена 1 корректировка: статус 'гражданский брак' заменен на 'женат / замужем'; в корректировке столбца с целями кредита нет необходимости в корректировках - далее цели будут категоризованы.

Появление дубликатов в данных возможно по причине повторных подач заявлений на кредит. При отклонении заявки в первый раз, через некоторое время (или сразу) можно подать ее повторно.

### Обработка аномалий

На всякий случай, проверим "на вшивость" все остальные столбцы, которые не анализировались на предыдущих шагах.

In [27]:
# проверка столбца children
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

-1 ребенок и сразу 20. Выведем количество строк логической индексацией по значению столбца `children`

In [28]:
# для случая -1 ребенок
data[data['children'] == -1].count()

children            47
days_employed       47
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        47
purpose             47
dtype: int64

In [29]:
#для случая 20 детей
data[data['children'] == 20].count()

children            76
days_employed       76
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        76
purpose             76
dtype: int64

Количество аномальных строк очень мало по сравнению с общим количеством в датасете. Для точности анализа исключим их из датасета.

In [30]:
#исключение аномалий в столбце children - перезапись переменной с данными и индексов
data = data[(data['children'] >= 0) & (data['children'] <= 5)].reset_index(drop=True)
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [31]:
# информация об обновленном датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21331 non-null  int64 
 1   days_employed     21331 non-null  int64 
 2   dob_years         21331 non-null  int64 
 3   education         21331 non-null  object
 4   education_id      21331 non-null  int64 
 5   family_status     21331 non-null  object
 6   family_status_id  21331 non-null  int64 
 7   gender            21331 non-null  object
 8   income_type       21331 non-null  object
 9   debt              21331 non-null  int64 
 10  total_income      21331 non-null  int64 
 11  purpose           21331 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [32]:
# проверка столбца education_id
data['education_id'].unique()

array([0, 1, 2, 3, 4])

По столбцу `education` порядок. Было 5 уникальных значений, и сейчас 5 уникальных id

In [33]:
# проверка столбца family_status_id
data['family_status_id'].value_counts()

0    12261
1     4134
4     2796
3     1189
2      951
Name: family_status_id, dtype: int64

По столбцу `family_status_id` порядок. Было 5 уникальных значений, и сейчас 5 уникальных id

In [34]:
# проверка столбца gender. Ожидаем 2 значения - F или M
data['gender'].value_counts()

F      14092
M       7238
XNA        1
Name: gender, dtype: int64

Непонятное значение XNA появляется в данных всего 1 раз. Заменим это значение на M, пропорция 2:1 не нарушится от одной замены.

In [35]:
# замена значения XNA
data['gender'] = data['gender'].replace('XNA', 'M')

In [36]:
# проверка замены
data['gender'].value_counts()

F    14092
M     7239
Name: gender, dtype: int64

In [37]:
# информация об обновленном датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21331 non-null  int64 
 1   days_employed     21331 non-null  int64 
 2   dob_years         21331 non-null  int64 
 3   education         21331 non-null  object
 4   education_id      21331 non-null  int64 
 5   family_status     21331 non-null  object
 6   family_status_id  21331 non-null  int64 
 7   gender            21331 non-null  object
 8   income_type       21331 non-null  object
 9   debt              21331 non-null  int64 
 10  total_income      21331 non-null  int64 
 11  purpose           21331 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [38]:
# проверка столбца debt
data['debt'].unique()

array([0, 1])

Со столбцом `debt` все в порядке

**Вывод по обработке дубликатов**

Датасет проверен на наличие аномалий в остальных столбцах.

Удалены строки со значением в столбце `children` -1 и 20 - не повлияет на точность анализа, также произведена 1 замена значения *XNA* в столбце `gender`

### Лемматизация

In [39]:
# лемматизация каждой строки в столбце purpose и добавление нового столбца

# создание функции для лемматизации
def lemmatizator(purpose):
    lemmas = m.lemmatize(purpose)                     
    
    for word in lemmas:
        if 'образован' in word:
            return 'образование'
        if 'авто' in word:
            return 'автомобиль'
        if 'жил' in word or 'недвиж' in word:
            return 'недвижимость'
        if 'свадьб' in word:
            return 'свадьба'

# добавление нового столбца
data['purpose_group'] = data['purpose'].apply(lemmatizator)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [40]:
# проверка количества уникальных значений в новом столбце purpose_group
data['purpose_group'].value_counts()

недвижимость    10751
автомобиль       4279
образование      3988
свадьба          2313
Name: purpose_group, dtype: int64

**Вывод по лемматизации**

Написана функция для лемматизации целей получения кредита, с помощью функции сразу получена категоризация.

### Категоризация данных

На текущем этапе произведем категоризацию по возрасту, уровню дохода и количеству детей в семье, с целью более удобной группировки при ответах на вопросы (шаг 3). Для этого напишем несколько новых функций и добавим новые столбцы методом .apply()

Дополнительно добавим новый столбец с переформулированным значением `debt` - для удобства в будущей группировке.

Для столбца `dob_years` произведем стандартную категоризацию (молодые и взрослые). Среди взрослых затем можно дополнительно группироваться по столбцу `income_type` для большей точности.

In [41]:
# функция для категоризации по возрасту
def dob_years_categories(dob_years):
    if dob_years <= 21:
        return 'молодые'
    return 'взрослые'

data['dob_years_group'] = data['dob_years'].apply(dob_years_categories)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,взрослые
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,взрослые
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,взрослые
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,взрослые
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,взрослые
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,взрослые
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,взрослые


In [42]:
# проверка по возрасту
data['dob_years_group'].value_counts()

взрослые    21056
молодые       275
Name: dob_years_group, dtype: int64

В случае с доходом, произведем категоризацию по такому принципу:
- Невысокий - не больше средней зар. платы по РФ;
- Средний - не больше 100 тыс. руб.
- Высокий - не больше 150 тыс. руб.;
- Очень высокий - все доходы выше 150 тыс. руб.

In [43]:
# функция для категоризации по уровню дохода
def total_income_categories(total_income):
    if total_income <= 54687:
        return 'невысокий'
    if total_income <= 100000:
        return 'средний'
    if total_income <= 150000:
        return 'высокий'
    else:
        return 'очень высокий'

# добавление категорий в новом столбце
data['total_income_group'] = data['total_income'].apply(total_income_categories)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,взрослые,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,взрослые,высокий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,взрослые,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,взрослые,очень высокий
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,взрослые,очень высокий
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,взрослые,очень высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,взрослые,очень высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,взрослые,высокий
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,взрослые,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,взрослые,высокий


In [44]:
# проверка по уровню дохода
data['total_income_group'].value_counts()

очень высокий    9135
высокий          7752
средний          3886
невысокий         558
Name: total_income_group, dtype: int64

В случае с количеством детей в семье, многодетными считаются семьи наличии 3 и более детей (исходя из того, что семьи с 3+ детьми могут претендовать на участок земли под строительство дома, помимо материнского капитала).

Категоризация будет следующей:
- Нет детей;
- 1-2 ребенка;
- Многодетная семья.

In [45]:
# функция для категоризации по количеству детей
def children_categories(children):
    if children == 0:
        return 'нет детей'
    if children <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетная семья'

data['children_group'] = data['children'].apply(children_categories)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,total_income_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,взрослые,очень высокий,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,взрослые,высокий,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,взрослые,высокий,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,взрослые,очень высокий,многодетная семья
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,взрослые,очень высокий,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,взрослые,очень высокий,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,взрослые,очень высокий,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,взрослые,высокий,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,взрослые,средний,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,взрослые,высокий,нет детей


In [46]:
# проверка по количеству детей
data['children_group'].value_counts()

нет детей            14091
1-2 ребенка           6860
многодетная семья      380
Name: children_group, dtype: int64

In [47]:
# функция для категоризации по факту возврата/не возврата кредита в срок
def debt_categories(debt):
    if debt == 0:
        return 'returned_on_time' # кредит возвращен вовремя
    return 'delay' # просрока возврата кредита


data['debt_group'] = data['debt'].apply(debt_categories)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,total_income_group,children_group,debt_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,взрослые,очень высокий,1-2 ребенка,returned_on_time
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,взрослые,высокий,1-2 ребенка,returned_on_time
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,взрослые,высокий,нет детей,returned_on_time
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,взрослые,очень высокий,многодетная семья,returned_on_time
4,0,12480,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,взрослые,очень высокий,нет детей,returned_on_time
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,взрослые,очень высокий,нет детей,returned_on_time
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,взрослые,очень высокий,нет детей,returned_on_time
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,взрослые,высокий,нет детей,returned_on_time
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,взрослые,средний,1-2 ребенка,returned_on_time
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,взрослые,высокий,нет детей,returned_on_time


**Вывод по категоризации**

Категоризация произведена, добавлены категории по 4 столбцам:
- `dob_years` - возраст клиентов
- `total_income` - уровень дохода
- `children` - количество детей в семье
- `debt` - для удобства при ответах на вопросы

## Изучение профилей клиентов

Критерием для сравнения категорий заемщиков между собой выступает вероятность невозврата кредита в срок.
- В каждом случае создана новая переменная для расчета вероятности по каждой строке сводной таблицы;
- Для форматирования вероятности в начале раздела создана функция, которая возвращает понятный формат результата;
- Форматированная вероятность добавляется в каждую сводную таблицу методом .apply();

При выводе значения вероятности невозврата каждая таблица сортируется по убыванию значения вероятности.

In [48]:
# создание функции для форматирования вероятности невозврата.
# будет применяться через метод .apply()

def probability_format(x):
    return '{:.1%}'.format(x)

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

При ответе на этот вопрос стоит производить группировку по наличию детей (`children_group`) и подсчетом количества случаев возврата/не возврата кредита (`debt`).

Напоминание по столбцу `debt`:
- 1 - имелась задолженность по возврату кредита;
- 0 - задолженность не имелась.

In [49]:
# создание сводной таблицы
q_1_pivot = data.pivot_table(index= 'children_group', 
                                    columns= 'debt_group', 
                                    values= 'debt', 
                                    aggfunc= 'count')

# объявление переменной delay_probability для расчета вероятности невозврата кредита в срок по каждой категории.
q_1_pivot_prob = q_1_pivot['delay'] / (q_1_pivot['delay'] + q_1_pivot['returned_on_time'])

# добавление значения вероятности и его форматирование
q_1_pivot['delay_probability'] = q_1_pivot_prob.apply(probability_format)

display(q_1_pivot.sort_values(by='delay_probability', ascending=False))
print('Всего заемщиков:', data.shape[0])

debt_group,delay,returned_on_time,delay_probability
children_group,,,
1-2 ребенка,638,6222,9.3%
многодетная семья,31,349,8.2%
нет детей,1063,13028,7.5%


Всего заемщиков: 21331


**Вывод**

Заемщики, не имеющие детей, менее склонны к просрочке кредита.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

При ответе на этот вопрос стоит производить группировку по семейному положению (`family_status`) и подсчетом количества случаев возврата/не возврата кредита (`debt`).

In [50]:
# создание сводной таблицы
q_2_pivot = data.pivot_table(index = 'family_status',
                                    columns = 'debt_group',
                                    values = 'debt',
                                    aggfunc = 'count')

# объявление переменной delay_probability для расчета вероятности невозврата кредита в срок по каждой категории.
q_2_pivot_prob = q_2_pivot['delay'] / (q_2_pivot['delay'] + q_2_pivot['returned_on_time'])

# добавление значения вероятности и его форматирование
q_2_pivot['delay_probability'] = q_2_pivot_prob.apply(probability_format)

display(q_2_pivot.sort_values(by='delay_probability', ascending=False))
print('Всего заемщиков:', data.shape[0])

debt_group,delay,returned_on_time,delay_probability
family_status,,,
не женат / не замужем,273,2523,9.8%
гражданский брак,385,3749,9.3%
женат / замужем,927,11334,7.6%
в разводе,84,1105,7.1%
вдовец / вдова,63,888,6.6%


Всего заемщиков: 21331


**Вывод**

Заемщики с узаконенными отношениями менее склонны к возврату кредита в срок (категории 'гражданский брак' и 'женат / замужем' в сумме дают больше значение вероятности). Самые ответственные категории - вдовцы / вдовы и разведенные.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

При ответе на этот вопрос стоит производить группировку по уровню дохода (`total_income_group`) и подсчетом количества случаев возврата/не возврата кредита (`debt`).

In [51]:
# создание сводной таблицы
q_3_pivot = data.pivot_table(index = 'total_income_group',
                                    columns = 'debt_group',
                                    values = 'debt',
                                    aggfunc = 'count')

# объявление переменной delay_probability для расчета вероятности невозврата кредита в срок по каждой категории.
q_3_pivot_prob = q_3_pivot['delay'] / (q_3_pivot['delay'] + q_3_pivot['returned_on_time'])

# добавление значения вероятности и его форматирование
q_3_pivot['delay_probability'] = q_3_pivot_prob.apply(probability_format)

display(q_3_pivot.sort_values(by='delay_probability', ascending=False))
print('Всего заемщиков:', data.shape[0])

debt_group,delay,returned_on_time,delay_probability
total_income_group,,,
высокий,656,7096,8.5%
средний,320,3566,8.2%
очень высокий,722,8413,7.9%
невысокий,34,524,6.1%


Всего заемщиков: 21331


**Вывод**

Заемщики с высоким и средним уровнем дохода менее склонны к возврату кредита в срок.

Самые ответственные - люди с невысоким уровнем дохода. При неизменности уровня дохода долгое время, для них весомым аргументом при получении новых кредитов будет хорошая кредитная история.

Что примечательно - люди с очень высоким уровнем дохода более ответственны, чем люди с высоким доходом. Возможно, это объясняется тем, что через кредиты очень богатые люди стремятся совершать большие покупки и постоянно увеличивать свое имущество.

### Как разные цели кредита влияют на его возврат в срок?

При ответе на этот вопрос стоит производить группировку по категориям целей кредита (`purpose_group`) и подсчетом количества случаев возврата/не возврата кредита (`debt`).

In [52]:
# создание сводной таблицы
q_4_pivot = data.pivot_table(index = 'purpose_group',
                                    columns = 'debt_group',
                                    values = 'debt',
                                    aggfunc = 'count')

# объявление переменной delay_probability для расчета вероятности невозврата кредита в срок по каждой категории.
q_4_pivot_prob = q_4_pivot['delay'] / (q_4_pivot['delay'] + q_4_pivot['returned_on_time'])

# добавление значения вероятности и его форматирование
q_4_pivot['delay_probability'] = q_4_pivot_prob.apply(probability_format)

display(q_4_pivot.sort_values(by='delay_probability', ascending=False))
print('Всего заемщиков:', data.shape[0])

debt_group,delay,returned_on_time,delay_probability
purpose_group,,,
автомобиль,400,3879,9.3%
образование,369,3619,9.3%
свадьба,183,2130,7.9%
недвижимость,780,9971,7.3%


Всего заемщиков: 21331


**Вывод**

Заемщики, получающие кредит под недвижимость, более ответственны в отношении кредитов. Недвижимость - очень важное приобретение, и при регулярной просрочке возможно лишиться ее (будет негде жить). Пока кредит (ипотека) не погашен, недвижимость принадлежит банку, а не заемщику.

Кредит под образование может браться с целью повышения квалификации и увеличения уровня дохода в будущем. Уровень дохода резко повышается не сразу после окончания обучения, поэтому просрочки объяснимы.

Наименее ответственны - автомобилисты.

## Общий вывод

Общий вывод можно сформулировать так:
- **Самый ответственный заемщик** - не имеет детей; не находится в законных отношениях, имеет средний или очень высокий уровень дохода, берет кредит с целью приобретения недвижимости.
- **Менее ответственный заемщик** - имеет детей, находится в законных отношениях, имеет относительно высокий уровень дохода, берет кредит под покупку автомобиля или образование.